In [1]:
# MVP

In [2]:
# import bibliotecas

from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression 

from unidecode import unidecode
from time import sleep
from datetime import datetime
from datetime import date
import os

import sqlite3

In [3]:
# estabelece conexão com banco de dados

con = sqlite3.connect("botbitarb.db")
cur = con.cursor()

In [4]:
# configurações webdriver

chrome_options = Options()
chrome_options.add_experimental_option(
    'excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')

In [5]:
# funções auxiliares

def preco_fmt(preco):
    preco = float(preco.replace('R','').replace('$','').replace('.','').replace(',','.').lstrip())
    return preco


In [6]:
# bot

class bitArbBot:

    def push(self):
        self.bot()

    def bot(self):
        
        valorBase = '199,82' # BRL
        
        # login
        self.driver = webdriver.Chrome('chromedriver', options=chrome_options)
        self.link = 'https://market.bitpreco.com/login/?'
        self.driver.get(self.link)
        sleep(5)
        self.driver.find_element(By.ID, "emailfield").send_keys("")
        self.driver.find_element(By.NAME, "pass").send_keys("")
        sleep(1)
        self.driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/main/div[2]/form/div[4]/button').click()
        sleep(5)
        self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
        sleep(1)
        
        # loop
        while True:
            try:
                self.link = 'https://market.bitpreco.com/trade/BTC-BRL'
                self.driver.get(self.link)
                sleep(5)
                self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
                sleep(60)
                # compra
                self.driver.find_element(By.XPATH,
                                         '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[1]/div[2]/div/div/div/button[1]/span[1]/span/span').click()
                sleep(1)
                # Linear regression ML model for predicting purchase price and estimating favorable or unfavorable condition for purchase 
                price_list = []
                favorable_condition = False
                while favorable_condition == False:
                    for prices in range(0, 5):
                        getPrice = self.driver.find_element(By.XPATH,
                                                           '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/div/div/input').get_attribute('value')
                        priceTemp = preco_fmt(getPrice)
                        price_list.append([datetime.now().strftime("%M"), priceTemp])
                        sleep(60)
                    data = pd.DataFrame(price_list, columns = ['min', 'price'])
                    X = data.iloc[:, 0:-1].values
                    y = data.iloc[:, -1].values
                    #price prediction
                    lin_reg = LinearRegression()
                    lin_reg.fit(X, y)
                    price_prediction = lin_reg.predict([[int(X[4])+1]])
                    #condition
                    priceNowTemp = self.driver.find_element(By.XPATH,
                                                           '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/div/div/input').get_attribute('value')
                    price_now = preco_fmt(priceNowTemp)
                    if price_now < price_prediction:
                        print('favorable condition')
                        price_list.clear()
                        del data
                        favorable_condition = True
                    else:
                        print('unfavorable condition')
                        price_list.clear()
                        del data
                        continue               
                # valor base 
                self.driver.find_element(By.XPATH, 
                                         '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[1]/div[2]/div/input').send_keys(valorBase)
                # pega melhor preço de compra unitário em float
                precoCompraTemp = self.driver.find_element(By.XPATH,
                                                           '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/div/div/input').get_attribute('value')
                precoCompra = preco_fmt(precoCompraTemp)
                # efetua ordem de compra
                self.driver.find_element(By.XPATH,
                                         '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/button/span[1]').click()
                timestamp = datetime.now()
                # imprime detalhes da transação de compra
                print()
                print('--- ordem executada ---')
                print('tipo: compra')
                print(f'preço de compra: {precoCompra}')
                print(f'valor base BRL {valorBase}')
                print(f'dia e hora: {timestamp}')
                print('-----------------------')
                print()
                # registra transação no banco de dados
                data = [
                    ("compra", precoCompra, "-", timestamp),
                ]
                cur.executemany("INSERT INTO transactions VALUES(?, ?, ?, ?)", data)
                con.commit()
                data.clear()

                # efetua ordem de venda condicionada a ganho em relação ao valor de compra
                tentativas = 0
                vendaExec = False
                while vendaExec == False:
                    try:
                        # venda
                        sleep(2)
                        self.driver.find_element(By.XPATH,
                                                 '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[1]/div[2]/div/div/div/button[2]/span[1]/span/span').click()
                        sleep(5)
                        # 100% saldo Bitcoin
                        self.driver.find_element(By.XPATH,
                                                 '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[1]/div[1]/div/div/div/div/p[4]/span[1]').click()                        
                        # pega melhor preço de venda unitário em float
                        precoVendaTemp = self.driver.find_element(By.XPATH,
                                                                  '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/div/div/input').get_attribute('value')
                        precoVenda = preco_fmt(precoVendaTemp)
                        # rentabilidade
                        rentab = ((precoVenda / precoCompra)-1)*100
                        # efetua ordem de venda condicionada a ganho
                        sleep(10)
                        if precoCompra < precoVenda and rentab >= 0.04:
                            self.driver.find_element(By.XPATH,
                                                     '//*[@id="root"]/div/div[1]/main/div[2]/div/div[2]/header/div[2]/div/section/form/div[2]/button/span[1]').click()                           
                            # imprime detalhes da transação de venda
                            timestamp = datetime.now()
                            print()
                            print('--- ordem executada ---')
                            print('tipo: venda')
                            print(f'preço de venda: {precoVenda}')
                            print(f'rentabilidade: {rentab}%')
                            print(f'dia e hora: {timestamp}')
                            print('-----------------------')
                            # registra transação no banco de dados
                            data = [
                                ("venda", precoVenda, rentab, timestamp),
                            ]
                            cur.executemany("INSERT INTO transactions VALUES(?, ?, ?, ?)", data)
                            con.commit()
                            data.clear()
                            vendaExec = True
                        else:
                            tentativas += 1
                            print(f'tentativa de venda {tentativas}')
                            print('atualizando preço de venda...')
                            sleep(20)
                    except NoSuchElementException:
                        sleep(5)
                        self.driver.find_element(By.XPATH, 
                                                 '//*[@id="root"]/div/div[1]/div/div/div/ul/div[2]/a[1]/div[1]/span').click()
                        sleep(2)
                        self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
                        continue
                    except ElementClickInterceptedException:
                        sleep(5)
                        self.driver.find_element(By.XPATH,
                                                 '/html/body/div[2]/div[2]/div[2]/button[2]/span[1]').click()
                        sleep(2)
                        self.driver.find_element(By.ID, "emailfield").send_keys("dan.tfarias@gmail.com")
                        self.driver.find_element(By.NAME, "pass").send_keys("Falabella@23")
                        sleep(1)
                        self.driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div/form/div[3]/button').click()
                        continue
            except NoSuchElementException:
                sleep(5)
                self.driver.find_element(By.XPATH, 
                                         '//*[@id="root"]/div/div[1]/div/div/div/ul/div[2]/a[1]/div[1]/span').click()
                sleep(2)
                self.driver.find_element(By.XPATH, '/html/body').send_keys(Keys.ESCAPE)
                continue


In [ ]:
start = bitArbBot()
start.push()

unfavorable condition
unfavorable condition
unfavorable condition
unfavorable condition
favorable condition


--- ordem executada ---
tipo: compra
preço de compra: 89710.0
valor base BRL 199,82
dia e hora: 2022-12-07 01:18:33.728197
-----------------------

tentativa de venda 1
atualizando preço de venda...
tentativa de venda 2
atualizando preço de venda...
tentativa de venda 3
atualizando preço de venda...
tentativa de venda 4
atualizando preço de venda...
tentativa de venda 5
atualizando preço de venda...
tentativa de venda 6
atualizando preço de venda...
tentativa de venda 7
atualizando preço de venda...
